# 20M Parameter Text Generation Model
## Training and Benchmarking Pipeline

This notebook implements a 20 million parameter transformer-based model for text generation and NLP tasks.

### 1. Setup and Dependencies

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets tokenizers accelerate wandb evaluate rouge-score nltk sacrebleu
!pip install sentencepiece protobuf

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
from transformers import get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np
import wandb
from tqdm.auto import tqdm
import json
import os

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

### 2. Model Configuration (10M Parameters)

In [ ]:
# Configure model to have ~10M parameters
config = GPT2Config(
    vocab_size=50257,
    n_positions=512,
    n_embd=256,
    n_layer=8,
    n_head=8,
    n_inner=1024,
    activation_function='gelu_new',
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
)

# Initialize model
model = GPT2LMHeadModel(config)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: {total_params * 4 / 1e6:.2f} MB (FP32)")

### 3. Data Preparation

In [ ]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Load dataset (using WikiText-103 for text generation)
dataset = load_dataset('wikitext', 'wikitext-103-v1')

print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Test samples: {len(dataset['test'])}")

In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=512,
        padding='max_length',
        return_tensors='pt'
    )

# Tokenize datasets
tokenized_train = dataset['train'].map(
    tokenize_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

tokenized_val = dataset['validation'].map(
    tokenize_function,
    batched=True,
    remove_columns=dataset['validation'].column_names
)

tokenized_train.set_format('torch')
tokenized_val.set_format('torch')

### 4. Training Configuration

In [ ]:
# Training hyperparameters
BATCH_SIZE = 16
LEARNING_RATE = 5e-4
EPOCHS = 3
WARMUP_STEPS = 500
GRADIENT_ACCUMULATION_STEPS = 4
MAX_GRAD_NORM = 1.0

# Create dataloaders
train_loader = DataLoader(tokenized_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(tokenized_val, batch_size=BATCH_SIZE)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
total_steps = len(train_loader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

print(f"Total training steps: {total_steps}")

### 5. Training Loop

In [ ]:
# Initialize wandb (optional)
# wandb.init(project='10m-text-generation', config={
#     'batch_size': BATCH_SIZE,
#     'learning_rate': LEARNING_RATE,
#     'epochs': EPOCHS,
#     'model_params': total_params
# })

def train_epoch(model, loader, optimizer, scheduler, device, epoch):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(loader, desc=f"Epoch {epoch}")
    
    for step, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()
        
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        progress_bar.set_postfix({'loss': loss.item() * GRADIENT_ACCUMULATION_STEPS})
    
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()
    
    avg_loss = total_loss / len(loader)
    perplexity = torch.exp(torch.tensor(avg_loss))
    return avg_loss, perplexity.item()

In [ ]:
# Training loop
best_val_loss = float('inf')
training_history = []

for epoch in range(1, EPOCHS + 1):
    print(f"\n{'='*50}")
    print(f"Epoch {epoch}/{EPOCHS}")
    print(f"{'='*50}")
    
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device, epoch)
    val_loss, val_perplexity = evaluate(model, val_loader, device)
    
    print(f"\nTrain Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val Perplexity: {val_perplexity:.2f}")
    
    training_history.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'val_perplexity': val_perplexity
    })
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_perplexity': val_perplexity
        }, 'best_model.pt')
        print("✓ Saved best model")

print("\n" + "="*50)
print("Training completed!")
print(f"Best validation loss: {best_val_loss:.4f}")

### 6. Text Generation

In [ ]:
def generate_text(prompt, max_length=100, temperature=0.8, top_k=50, top_p=0.95):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test generation
prompts = [
    "The future of artificial intelligence",
    "In a world where technology",
    "Scientists have discovered"
]

print("\n" + "="*50)
print("Text Generation Examples")
print("="*50)

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("-" * 50)
    generated = generate_text(prompt, max_length=150)
    print(generated)
    print()

### 7. Benchmarking and Evaluation

In [ ]:
import time
from evaluate import load

# Load metrics
perplexity_metric = load('perplexity', module_type='metric')

def benchmark_model(model, test_loader, device):
    model.eval()
    
    # Inference speed
    start_time = time.time()
    total_tokens = 0
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(test_loader, desc="Benchmarking")):
            if i >= 100:  # Test on 100 batches
                break
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            total_tokens += input_ids.numel()
    
    elapsed_time = time.time() - start_time
    tokens_per_second = total_tokens / elapsed_time
    
    return {
        'tokens_per_second': tokens_per_second,
        'inference_time': elapsed_time,
        'total_tokens': total_tokens
    }

# Run benchmarks
test_loader = DataLoader(tokenized_val, batch_size=BATCH_SIZE)
benchmark_results = benchmark_model(model, test_loader, device)

print("\n" + "="*50)
print("Benchmark Results")
print("="*50)
print(f"Tokens per second: {benchmark_results['tokens_per_second']:.2f}")
print(f"Inference time: {benchmark_results['inference_time']:.2f}s")
print(f"Total tokens processed: {benchmark_results['total_tokens']:,}")

In [ ]:
# Final evaluation metrics
final_val_loss, final_perplexity = evaluate(model, val_loader, device)

metrics_summary = {
    'model_parameters': total_params,
    'final_val_loss': final_val_loss,
    'final_perplexity': final_perplexity,
    'tokens_per_second': benchmark_results['tokens_per_second'],
    'training_epochs': EPOCHS,
    'best_val_loss': best_val_loss
}

print("\n" + "="*50)
print("Final Metrics Summary")
print("="*50)
for key, value in metrics_summary.items():
    print(f"{key}: {value}")

# Save metrics
with open('metrics_summary.json', 'w') as f:
    json.dump(metrics_summary, f, indent=2)

### 8. Save Model

In [ ]:
# Save final model
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

# Save to Google Drive (optional)
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r ./final_model /content/drive/MyDrive/10m_text_model

print("Model saved successfully!")